In [1]:
import os, sys
sys.path.append("./beaconrunner")

import numpy as np
import pandas as pd

import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

from experiments.run import run
import experiments.templates.rl.experiment as rl_experiment

c:\users\shyam\documents\github\beaconrunnerv2\env\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [3]:
from stable_baselines3.common.env_checker import check_env

env = rl_experiment.beacon_env
check_env(env)

2022-01-03 20:12:37,191 - root - INFO - Starting simulation 0 / run 0 / subset 0
8 proposing block for slot 1
2022-01-03 20:12:37,475 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:12:37,729 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:12:37,836 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:12:37,944 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:12:38,053 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:12:38,171 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:12:38,295 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:12:38,417 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:12:38,532 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:12:38,653 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:12:38,763 - root - INFO - Starting simulation 0 / run 0 / subset 0

In [ ]:
from stable_baselines3 import A2C

model = A2C("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=25000)

obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
2022-01-03 20:20:50,656 - root - INFO - Starting simulation 0 / run 0 / subset 0
8 proposing block for slot 1
2022-01-03 20:20:50,955 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:20:51,212 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:20:51,326 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:20:51,436 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:20:51,549 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:20:51,659 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:20:51,770 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:20:51,879 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:20:52,000 - root - INFO - Starting simulation 0 / run 0 / subset 0
2022-01-03 20:20:52,109 - root - INFO - Starting simulation 0 / run